<a href="https://colab.research.google.com/github/JUANCARLOST-LAB/Cancer-Prediction/blob/main/Type_of_Cancer_Prediction_with_Logistic_Regression_from_scratch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Type of Cancer Prediction

In this project I will build a Logistic Regression Model from scratch to predict the type of cancer a person has, either benign or malign, I will implement each function needed from scratch, using numpy library to use vectorized functions to make the operations faster, I will be working with the dataset provided on the following link: https://www.kaggle.com/datasets/erdemtaha/cancer-data

First I will build all the functions needed for the Logistic Regression Model, then I will process the data, to treat the null values in case of needed and also i will normalize the data for the algorithm to work better, then I will predict the data and measure some metrics to see how good the model is.

# Logistic Regression

## Import Libraries

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
# Function to initialize w and b
def initialize(dims):
  w = np.zeros((dims, 1), dtype=float)
  b = 0.0
  return w, b

## Sigmoid function
I will implement the following function using numpy:

$\sigma(x)=\frac{1}{1 + e^{-x}}$

In [ ]:
def sigmoid(x):
  # Uses numpy to calculate the sigmoid function
  s = 1 / (1 + np.exp(-x))
  return s

## Cost Function
Function that receives parameters $w, b, X, Y$ and calculates the cost function $J(w,b)$ by calculating the average of the loss functions.

The loss (error) function is calculated with the following formula


$L(\hat{y}, y) = -(y\log(\hat{y}) + (1-y)\log(1 - \hat{y}))$

 where $\hat y$ is the predicted value and $y$ is the real value


 The cost function is computed as follow

$J(w,b) = -\frac{1}{m}\sum\limits_{i=1}^m L(\hat{y}^{(i)}, y^{(i)}) = -\frac{1}{m}\sum\limits_{i=1}^m(y^{(i)}log\hat{y}^{(i)} + (1-y)log(1 - \hat{y}^{(i)}))$

In [ ]:
# X.shape is supposed to be (nx, m)
def cost_function(w, b, X, Y):
  # m = number of samples
  m = X.shape[1]
  # Calculate yhat (A)
  A = sigmoid(np.dot(w.T, X)+b)
  # Calculate the cost function
  cost = -1/m * (np.dot(Y.T, np.log(np.maximum(A,0.0000001)).T) + np.dot((1-Y).T, np.log(np.maximum(1-A,0.0000001)).T))
  return cost

## Gradient Descent Computation

In this function I will calculate $dw$ and $db$ that will be used to update the values of w and b, also I will calculate the actual cost function value.

The function will receive the parameters w, b, X and Y.

In [ ]:
def gradient_descent(w, b, X, Y):
  # m: number of samples
  m = X.shape[1]

  # Calculate A: predicted values with currently values of w and b
  A = sigmoid(np.dot(w.T, X) + b) # w.shape = (30,1)

  # Calculate value of dZ
  dZ = A - Y.T

  # Calculate value of dw
  dw = 1/m * np.dot(X, dZ.T)

  # Calculate value of db
  db = 1/m * np.sum(dZ)

  # Calculate cost function value
  cost = cost_function(w, b, X, Y)

  # Create dictionary containing values of db and dw
  params = {
      "db": db,
      "dw": dw
  }

  return cost, params

## Optmizer function

In this function I will run the iterations such that the values of w and b will be getting updated, every 150 iterations I will print the cost function values to see how the model is improving. For these functions I will receive the parameters $w, b, X, Y, numiterations, learningrate$, num_iterations parameter will contain the number of iterations and learning_rate will determine the change of w and b, by using the following formula

$b:= b - \alpha db$ and $w:=w - \alpha dw$ where $\alpha$ is the learning rate.

In [ ]:
def optimize(w, b, X, Y, num_iterations = 2000, learning_rate = 0.001):
  for i in range(num_iterations):
    cost, params = gradient_descent(w, b, X, Y)

    dw = params["dw"]
    db = params["db"]

    w = w - learning_rate * dw
    b = b - learning_rate * db

    if i%150 == 0:
      print(f"Cost function value: {cost}")

  return w, b, cost_function(w, b, X, Y)

## Train function

In this function I will receive parameters X_train, y_train, num_iterations and learning_rate to train the model

In [ ]:
def train(X_train, y_train, num_iterations = 2000, learning_rate = 0.001):
  # m: number of features
  m = X_train.shape[1]
  print(type(X_train))
  # Extract only values of X_train and y_train
  if type(X_train) != np.ndarray:
    X_train_values = X_train.values.reshape((-1, m)).T
  else:
    X_train_values = X_train.T
  y_train_values = y_train.values.reshape((1,-1)).T

  #X_train_values.shape = (30, 455)
  w, b = initialize(m)
  w, b, cost = optimize(w, b, X_train_values, y_train_values, num_iterations, learning_rate)

  print(f'Cost function value after training: {cost}')
  return w, b

## Predict Function

Function that given input x_test produces a y_prediction

In [ ]:
def predict(w, b, X_test):
  m = X_test.shape[1]
  # Extract only values of X_test
  if type(X_test) != np.ndarray:
    X_test_values = X_test.values.reshape((-1,m)).T
  else:
    X_test_values = X_test.T
  y_pred = sigmoid(np.dot(w.T, X_test_values) + b).reshape((1,-1))

  for i in range(y_pred.shape[1]):
    # If probability is greater than 0.5 then it assigns it with 1 (malign tumor)
    if y_pred[0, i]>0.5:
      y_pred[0,i] = 1

    # Otherwise, if it is less than or equal to 0.5 it assigns it with 0 (benign tumor)
    else:
      y_pred[0,i] = 0

  # Return the predictions
  return y_pred

## Accuracy function

Function that receives to parameters, y_pred and y_test which are the predicted and real values, it will return the accuracy value

In [ ]:
def accuracy(y_pred, y_test):
  #Obtain only values of y_test
  y_test_values = y_test.values.reshape((1,-1)).T

  # It predicts correcty when y_pred and y_test are equal, both are 0 or both are 1
  # Through dot product I am able to calculate these values
  acc = np.dot(y_pred, y_test_values) + np.dot(1 - y_pred, (1 - y_test_values))

  return 100 * acc / y_pred.shape[1]

# Data Wrangling
In these section I will load the dataset and process it to pass it to the logisic regression model and make the prediction

In [ ]:
url = '/content/drive/MyDrive/Datasets/Cancer_Data.csv'

# Load dataset
df_cancer = pd.read_csv(url, sep = ',')

df_cancer.head()

,id,diagnosis,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,...,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst,Unnamed: 32
0,842302,M,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,...,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890,NaN
1,842517,M,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,...,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902,NaN
2,84300903,M,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,...,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758,NaN
3,84348301,M,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,...,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300,NaN
4,84358402,M,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,...,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678,NaN


In [ ]:
# Information of columns
df_cancer.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 569 entries, 0 to 568
Data columns (total 33 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   id                       569 non-null    int64  
 1   diagnosis                569 non-null    object 
 2   radius_mean              569 non-null    float64
 3   texture_mean             569 non-null    float64
 4   perimeter_mean           569 non-null    float64
 5   area_mean                569 non-null    float64
 6   smoothness_mean          569 non-null    float64
 7   compactness_mean         569 non-null    float64
 8   concavity_mean           569 non-null    float64
 9   concave points_mean      569 non-null    float64
 10  symmetry_mean            569 non-null    float64
 11  fractal_dimension_mean   569 non-null    float64
 12  radius_se                569 non-null    float64
 13  texture_se               569 non-null    float64
 14  perimeter_se             5

**Insights**

Doesn't contains null data,except for *Unnamed: 32* column which I will delete, I will also delete id columns because it doesn't gives us information

In [ ]:
# Delete unnecesary columns
df_cancer_copy = df_cancer.copy()

df_cancer.drop(columns = ['id', 'Unnamed: 32'], inplace = True)

In [ ]:
df_cancer.head()

,diagnosis,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,symmetry_mean,...,radius_worst,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst
0,M,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,0.2419,...,25.38,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890
1,M,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,0.1812,...,24.99,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902
2,M,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,0.2069,...,23.57,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758
3,M,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,0.2597,...,14.91,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300
4,M,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,0.1809,...,22.54,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678


## Diagnois columns

This will be our target columns, it contains the veredict of either the cancer is benign or malign

In [ ]:
# Amount of each value
df_cancer.diagnosis.value_counts()

B    357
M    212
Name: diagnosis, dtype: int64

Our dataset contains 357 cases of benign cancer (B) and 212 cases of malign cancer (M), this is a categorical column, so I will transform B into 0's and M into 1's

In [ ]:
df_cancer['diagnosis'] = df_cancer['diagnosis'].map({'M': 1, 'B':0})
df_cancer.head()

,diagnosis,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,symmetry_mean,...,radius_worst,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst
0,1,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,0.2419,...,25.38,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890
1,1,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,0.1812,...,24.99,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902
2,1,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,0.2069,...,23.57,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758
3,1,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,0.2597,...,14.91,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300
4,1,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,0.1809,...,22.54,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678


In [ ]:
df_cancer.diagnosis.value_counts()

0    357
1    212
Name: diagnosis, dtype: int64

## Test Model

In [ ]:
# Divide data into train and test
from sklearn.model_selection import train_test_split

# Extract features from DataFrame
X = df_cancer.drop(columns=['diagnosis'])
y = df_cancer['diagnosis']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42)

In [ ]:
w, b = train(X_train, y_train, learning_rate = 0.000001)

<class 'pandas.core.frame.DataFrame'>
Cost function value: [[0.69314718]]
Cost function value: [[0.61118451]]
Cost function value: [[0.5662195]]
Cost function value: [[0.53394134]]
Cost function value: [[0.50886739]]
Cost function value: [[0.48844961]]
Cost function value: [[0.47129408]]
Cost function value: [[0.45655176]]
Cost function value: [[0.44366612]]
Cost function value: [[0.43225251]]
Cost function value: [[0.42203413]]
Cost function value: [[0.41280534]]
Cost function value: [[0.40440935]]
Cost function value: [[0.39672402]]
Cost function value after training: [[0.39430281]]


In [ ]:
y_pred = predict(w, b, X_test)

acc = accuracy(y_pred, y_test)
print(f'Accuracy score without standarization: {acc[0,0]}')

Accuracy score without standarization: 93.85964912280701


**Conclusion**

This model gives us an accuracy of 93.8%

Now we will train the model after normalizing the data

In [ ]:
# Divide data into train and test
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Extract features from DataFrame
X = df_cancer.drop(columns=['diagnosis'])
y = df_cancer['diagnosis']

# Split data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42)

# Standarize data
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)


In [ ]:
w, b = train(X_train, y_train, num_iterations=2500 ,learning_rate = 0.0001)

<class 'numpy.ndarray'>
Cost function value: [[0.69314718]]
Cost function value: [[0.66519909]]
Cost function value: [[0.6397409]]
Cost function value: [[0.61651559]]
Cost function value: [[0.59528261]]
Cost function value: [[0.57582256]]
Cost function value: [[0.55793897]]
Cost function value: [[0.54145798]]
Cost function value: [[0.52622689]]
Cost function value: [[0.51211205]]
Cost function value: [[0.49899673]]
Cost function value: [[0.48677889]]
Cost function value: [[0.47536928]]
Cost function value: [[0.46468963]]
Cost function value: [[0.45467118]]
Cost function value: [[0.44525331]]
Cost function value: [[0.43638247]]
Cost function value after training: [[0.4307488]]


In [ ]:
y_pred = predict(w, b, X_test)

acc = accuracy(y_pred, y_test)

print(f'Accuracy score with data standarized: {acc[0,0]}')

Accuracy score with data standarized: 96.49122807017544


**Conclusion**
This gave us an accuracy of 96.5%, slightly better than without standarazing.

## Sklearn Logistic Regression Model

Now we will use the model provided by scikit-learn library

In [ ]:
# Import library
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn import preprocessing

lr = LogisticRegression(max_iter = 2500)

# Extract features from DataFrame
X = df_cancer.drop(columns=['diagnosis'])
y = df_cancer['diagnosis']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42)
scaler = preprocessing.StandardScaler()

X_train = scaler.fit_transform(X_train)

# Train model
lr.fit(X_train, y_train)

# Predict
X_test = scaler.transform(X_test)
y_pred = lr.predict(X_test)

# Accuracy score
acc = accuracy_score(y_test, y_pred)

print(f'Accuracy score with sklearn library model: {acc}')

Accuracy score with sklearn library model: 0.9736842105263158
